# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_val using StandardScaler

scaler = StandardScaler()

# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X.columns, index=X_val.index)



In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Calculate a baseline r-squared score on training data
train_r2 = model.score(X_train_scaled, y_train)
val_r2 = model.score(X_val_scaled, y_val)

print(f"Baseline Training R-squared: {train_r2:.4f}")
print(f"Baseline Validation R-squared: {val_r2:.4f}")

Baseline Training R-squared: 0.7868
Baseline Validation R-squared: 0.6376


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [8]:
# Your code here

# Set up data structure
interaction_results = []

# Find combinations of columns and loop over them
for col1, col2 in combinations(X_train.columns, 2):
    
    # Make copies of X_train and X_val
    X_train_copy = X_train.copy()
    X_val_copy = X_val.copy()
    
    # Add interaction term to data
    X_train_copy[f"{col1}_{col2}"] = X_train_copy[col1] * X_train_copy[col2]
    X_val_copy[f"{col1}_{col2}"] = X_val_copy[col1] * X_val_copy[col2]

    
    # Find r-squared score (fit on training data, evaluate on validation data)
    
    # Append to data structure
    model.fit(X_train_copy, y_train)
    r2_val = model.score(X_val_copy, y_val)

    interaction_results.append((col1, col2, r2_val))
    
# Sort and subset the data structure to find the top 7
interaction_results.sort(key=lambda x: x[2], reverse=True)
top_interactions = interaction_results[:7]

print("Top 7 interactions:")
for interaction in top_interactions:
    print(f"{interaction[0]} * {interaction[1]}: R-squared = {interaction[2]:.4f}")

Top 7 interactions:
LotArea * 1stFlrSF: R-squared = 0.7211
LotArea * TotalBsmtSF: R-squared = 0.7072
LotArea * GrLivArea: R-squared = 0.6691
LotArea * Fireplaces: R-squared = 0.6530
2ndFlrSF * TotRmsAbvGrd: R-squared = 0.6473
OverallCond * TotalBsmtSF: R-squared = 0.6429
OverallQual * 2ndFlrSF: R-squared = 0.6422


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [10]:
# Your code here

# Loop over top 7 interactions

    # Extract column names from data structure

    # Construct new column name
    
    # Add new column to X_train and X_val
for col1, col2, _ in top_interactions:
    X_train_scaled[f"{col1}_{col2}"] = X_train[col1] * X_train[col2]
    X_val_scaled[f"{col1}_{col2}"] = X_val[col1] * X_val[col2]

print(f"Features after adding interactions: {X_train_scaled.columns.tolist()}")


Features after adding interactions: ['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces', 'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea', 'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd', 'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF']


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [18]:
# Your code here

# Set up data structure
polynomial_results = []

# Loop over all columns
for col in X_train.columns:
    for degree in [2, 3, 4]:
        X_train_copy = X_train[[col]].copy()
        X_val_copy = X_val[[col]].copy()

    # Loop over degrees 2, 3, 4
        
        # Make a copy of X_train and X_val
    
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_train_poly = poly.fit_transform(X_train_copy)
        X_val_poly = poly.transform(X_val_copy)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        if hasattr(poly, 'get_feature_names_out'):
            poly_features = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out([col]), index=X_train.index)
            poly_features_val = pd.DataFrame(X_val_poly, columns=poly.get_feature_names_out([col]), index=X_val.index)
        else:
            poly_features = pd.DataFrame(X_train_poly, columns=poly.get_feature_names(), index=X_train.index)
            poly_features_val = pd.DataFrame(X_val_poly, columns=poly.get_feature_names(), index=X_val.index)

        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        X_train_poly_final = pd.concat([X_train_scaled, poly_features], axis=1).drop(columns=[col])
        X_val_poly_final = pd.concat([X_val_scaled, poly_features_val], axis=1).drop(columns=[col])

        # Find r-squared score on validation
        # Append to data structure
        model.fit(X_train_poly_final, y_train)
        r2_val = model.score(X_val_poly_final, y_val)

        polynomial_results.append((col, degree, r2_val))
# Sort and subset the data structure to find the top 7
polynomial_results.sort(key=lambda x: x[2], reverse=True)
top_polynomials = polynomial_results[:7]

print("Top 7 polynomials:")
for poly in top_polynomials:
    print(f"{poly[0]}^{poly[1]}: R-squared = {poly[2]:.4f}")

Top 7 polynomials:
GrLivArea^3: R-squared = 0.8283
OverallQual^3: R-squared = 0.8068
OverallQual^4: R-squared = 0.8033
OverallQual^2: R-squared = 0.8026
2ndFlrSF^3: R-squared = 0.7768
2ndFlrSF^4: R-squared = 0.7697
LotArea^3: R-squared = 0.7654


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [22]:
# Your code here

# Filter out duplicates

# Loop over remaining results

    # Create polynomial terms
    
    # Concat new polynomials to X_train and X_val
    
used_columns = set()
for col, degree, _ in top_polynomials:
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train[[col]])
    X_val_poly = poly.transform(X_val[[col]])

    try:
            # Try using get_feature_names_out (scikit-learn >= 1.0)
            poly_features = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out([col]), index=X_train.index)
            poly_features_val = pd.DataFrame(X_val_poly, columns=poly.get_feature_names_out([col]), index=X_val.index)
    except AttributeError:
            # Fallback to get_feature_names (older versions)
            poly_features = pd.DataFrame(X_train_poly, columns=poly.get_feature_names(), index=X_train.index)
            poly_features_val = pd.DataFrame(X_val_poly, columns=poly.get_feature_names(), index=X_val.index)

    for feature in poly_features.columns:
        if feature not in used_columns:
            X_train_scaled[feature] = poly_features[feature]
            X_val_scaled[feature] = poly_features_val[feature]
            used_columns.add(feature)


    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [23]:
# Your code here
print(f"Features after adding polynomials: {X_train_scaled.columns.tolist()}")

Features after adding polynomials: ['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces', 'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea', 'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd', 'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF', 'x0', 'x0^2', 'x0^3', 'x0^4']


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [24]:
# Your code here

It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [25]:
# Your code here
for n_features in [5, 10, 15, 20]:
    selector = RFE(estimator=model, n_features_to_select=n_features)
    selector.fit(X_train_scaled, y_train)

Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [28]:
# Your code here
train_r2_rfe = selector.score(X_train_scaled, y_train)
val_r2_rfe = selector.score(X_val_scaled, y_val)
    
print(f"RFE with {n_features} features - Training R-squared: {train_r2_rfe:.4f}, Validation R-squared: {val_r2_rfe:.4f}, Features: {n_features}")

RFE with 20 features - Training R-squared: 0.8298, Validation R-squared: 0.8155, Features: 20


Compare the results. Which features would you choose to use?

In [29]:
# Your written answer here
for alpha in [0.1, 1, 10, 100]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    
    # R-squared scores and number of features
    train_r2_lasso = lasso.score(X_train_scaled, y_train)
    val_r2_lasso = lasso.score(X_val_scaled, y_val)
    num_features = np.sum(lasso.coef_ != 0)
print(f"Lasso with alpha={alpha} - Training R-squared: {train_r2_lasso:.4f}, Validation R-squared: {val_r2_lasso:.4f}, Features: {num_features}")

Lasso with alpha=100 - Training R-squared: 0.8358, Validation R-squared: 0.8564, Features: 18


## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [31]:
# Your code here
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

for col1, col2, _ in top_interactions:
    X_test_scaled[f"{col1}_{col2}"] = X_test[col1] * X_test[col2]

for col, degree, _ in top_polynomials:
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_test_poly = poly.transform(X_test[[col]])
    poly_features_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names_out([col]))
    X_test_scaled = pd.concat([X_test_scaled, poly_features_test], axis=1)
    
if hasattr(poly, 'get_feature_names_out'):
    poly_features_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names_out([col]), index=X_test.index)
else:
    poly_features_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names(), index=X_test.index)


# Fit model on complete train + validation set
X_combined = pd.concat([X_train_scaled, X_val_scaled])
y_combined = pd.concat([y_train, y_val])
model.fit(X_combined, y_combined)

NotFittedError: This PolynomialFeatures instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [32]:
# Your code here
test_r2 = model.score(X_test_scaled, y_test)
test_mse = mean_squared_error(y_test, model.predict(X_test_scaled))

print(f"Test R-squared: {test_r2:.4f}")
print(f"Test Mean Squared Error: {test_mse:.4f}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.